# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\\$645 - 675) underperformed compared to schools with smaller budgets (\\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

**Note:**
Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [430]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import sqlite3

# loading data from python.db database
con = sqlite3.connect('data/python.db')

# Read 'school' and 'student' tables into pandas dataframe
school_data = pd.read_sql_query("SELECT * FROM school", con)
student_data = pd.read_sql_query("SELECT * FROM student", con)


In [431]:
# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,id_x,Student ID,student_name,gender,grade,school_name,reading_score,math_score,id_y,School ID,type,size,budget
0,0,0,Paul Bradley,M,9th,Huang High School,66,79,0,0,District,2917,1910635
1,1,1,Victor Smith,M,12th,Huang High School,94,61,0,0,District,2917,1910635
2,2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,0,District,2917,1910635
3,3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,0,District,2917,1910635
4,4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [433]:
# Create a District Summary


In [434]:
# Total number of schools

query = "SELECT COUNT(DISTINCT(school_name)) as total_schools FROM school"
total_schools = con.execute(query).fetchone()[0]
print(f'Total Schools: {total_schools}')



Total Schools: 15


In [435]:
# Total number of students

query = "SELECT SUM(DISTINCT(size)) as total_students FROM school"
total_students = con.execute(query).fetchone()[0]
print(f'Total Students: {total_students}')




Total Students: 39170


In [436]:
# Total budget

query = "SELECT SUM(DISTINCT(budget)) as total_budget FROM school"
total_budget = con.execute(query).fetchone()[0]
print(f'Total Budget: {total_budget}')



Total Budget: 24649428


In [437]:
# Average math score

query = "SELECT AVG(math_score) as avg_math_score FROM student"
avg_math_score = con.execute(query).fetchone()[0]
print(f'Average Math Score: {avg_math_score:.2f}')



Average Math Score: 78.99


In [438]:
# Average reading score
query = "SELECT AVG(reading_score) as avg_reading_score FROM student"
avg_reading_score = con.execute(query).fetchone()[0]
print(f'Average Reading Score: {avg_reading_score:.2f}')



Average Reading Score: 81.88


In [439]:
# Overall average score

query = "SELECT ((AVG(reading_score) + AVG(math_score))/2) as avg_score FROM student"
avg_score = con.execute(query).fetchone()[0]
print(f'Average Score: {avg_score:.2f}')



Average Score: 80.43


In [440]:
# Percentage of passing math (70 or greater)

query = """
SELECT 
((((COUNT(CASE WHEN math_score >= 70 THEN 1 END)*1.0)/(SELECT SUM(DISTINCT(size)) FROM school)*100) +
((COUNT(CASE WHEN reading_score >= 70 THEN 1 END)*1.0)/(SELECT SUM(DISTINCT(size)) FROM school)*100) )/2) AS avg_passing_rate
FROM student

"""
cur = con.cursor()
cur.execute(query)
result = cur.fetchone()
if result and result[0] is not None:
    avg_passing_rate = result[0]
    print(f"Average Passing Rate: {avg_passing_rate:.2f}%")

# Using both the passing rate of math and english and averaging them to use in 1.3.1.1

Average Passing Rate: 80.39%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

### Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [445]:
#  Sort and display the top five schools in overall passing rate

query = """
WITH top_5_schools AS 
(SELECT school_name, 
    ((((COUNT(CASE WHEN math_score >= 70 THEN 1 END) * 1.0) / (SELECT SUM(size) FROM school WHERE school_name = student.school_name) * 100) +
    ((COUNT(CASE WHEN reading_score >= 70 THEN 1 END) * 1.0) / (SELECT SUM(size) FROM school WHERE school_name = student.school_name) * 100)) / 2) 
    AS passing_rate
    FROM student
    GROUP BY school_name
    ORDER BY passing_rate DESC
    LIMIT 5)
SELECT 
school_name AS school_name, 
((((COUNT(CASE WHEN math_score >= 70 THEN 1 END)*1.0)/ (SELECT SUM(size) FROM school WHERE school_name = student.school_name) * 100) +
((COUNT(CASE WHEN reading_score >= 70 THEN 1 END)*1.0)/ (SELECT SUM(size) FROM school WHERE school_name = student.school_name) * 100) )/2) as passing_rate
FROM student
GROUP BY school_name 
ORDER BY passing_rate DESC
LIMIT 5
"""
passing_rates = con.execute(query).fetchall()
for school in passing_rates:
    print(f'School: {school[0]}, Passing Rate: {school[1]:.2f}%')


School: Cabrera High School, Passing Rate: 95.59%
School: Thomas High School, Passing Rate: 95.29%
School: Pena High School, Passing Rate: 95.27%
School: Griffin High School, Passing Rate: 95.27%
School: Wilson High School, Passing Rate: 95.20%


In [446]:
# Calculate per student budget
query = """
WITH queryA AS
(SELECT SUM(DISTINCT(school.budget)) as total_budget
FROM school
WHERE school.school_name IN 
    (SELECT school_name 
    FROM student
    GROUP BY school_name 
    ORDER BY ((COUNT(CASE WHEN math_score >= 70 THEN 1 END)*1.0)/(SELECT SUM(DISTINCT(size)) as total_students FROM school)* 100)
    LIMIT 5))
SELECT ((queryA.total_budget)/(SUM(DISTINCT(school.size)))) as per_student_budget
FROM school, queryA
WHERE school.school_name IN 
(SELECT school_name 
    FROM student
    GROUP BY school_name 
    ORDER BY ((COUNT(CASE WHEN math_score >= 70 THEN 1 END)*1.0)/(SELECT SUM(DISTINCT(size)) as total_students FROM school)* 100)
    LIMIT 5)

"""
cur = con.cursor()
cur.execute(query)
result = cur.fetchone()
if result and result[0] is not None:
    print(f"Per Student Budget: {result[0]}")

Per Student Budget: 615


In [447]:
# Cacluate the avg math and reading score
query = """
WITH queryA AS
(SELECT SUM(DISTINCT(school.budget)) as total_budget
FROM school
WHERE school.school_name IN 
    (SELECT school_name 
    FROM student
    GROUP BY school_name 
    ORDER BY ((COUNT(CASE WHEN math_score >= 70 THEN 1 END)*1.0)/(SELECT SUM(DISTINCT(size)) as total_students FROM school)* 100)
    LIMIT 5))
SELECT AVG(math_score) as avg_math_score , AVG(reading_score) as avg_reading_score 
FROM student
"""
cur = con.cursor()
cur.execute(query)
result = cur.fetchone()
if result and result[0] is not None:
    avg_math_score, avg_reading_score = result
    print(f"Average Math Score: {avg_math_score:.2f}")
    print(f"Average Reading Score: {avg_reading_score:.2f}")

Average Math Score: 78.99
Average Reading Score: 81.88


In [448]:
# Calculate total school budget
query = """
SELECT SUM(DISTINCT(school.budget)) as total_budget
FROM school
WHERE school.school_name IN 
    (SELECT school_name 
    FROM student
    GROUP BY school_name 
    ORDER BY ((COUNT(CASE WHEN math_score >= 70 THEN 1 END)*1.0)/(SELECT SUM(DISTINCT(size)) as total_students FROM school)* 100)
    LIMIT 5)
"""
cur = con.cursor()
cur.execute(query)
result = cur.fetchone()
if result and result[0] is not None:
    print(f"Total Budget: {result[0]}")


Total Budget: 3851175


#### Find the passing rate for math and reading (above 70 points)

In [683]:
# Find the total counts of math result
#(SELECT SUM(size) FROM school WHERE school_name = student.school_name)

# Find the counts for math result in each school that pass 70 or higher
#COUNT(CASE WHEN math_score >= 70 THEN 1 END) * 1.0) / (SELECT SUM(size) FROM school WHERE school_name = student.school_name) * 100)


# Calculate the math passing rate
query = """
SELECT ((COUNT(CASE WHEN math_score >= 70 THEN 1 END) * 1.0) / (SELECT SUM(DISTINCT(size)) FROM school)) * 100 AS math_passing_rate
FROM student
JOIN school ON school.school_name = student.school_name
"""
math_passing_rate = con.execute(query).fetchall()
for row in math_passing_rate:
    print(f'Math Passing Rate: {row[0]:.2f}%')


Math Passing Rate: 74.98%


In [699]:
# Find the total counts of read result
#(SELECT SUM(size) FROM school WHERE school_name = student.school_name)

# Find the counts for read result in each school that pass 70 or higher
#COUNT(CASE WHEN reading_score >= 70 THEN 1 END) * 1.0) / (SELECT SUM(size) FROM school WHERE school_name = student.school_name) * 100)

# Calculate the read passing rate
query = """
SELECT ((COUNT(CASE WHEN reading_score >= 70 THEN 1 END) * 1.0) / (SELECT SUM(DISTINCT(size)) FROM school)) * 100 AS reading_passing_rate
FROM student
JOIN school ON school.school_name = student.school_name
"""
reading_passing_rate = con.execute(query).fetchall()
for row in reading_passing_rate:
    print(f'Reading Passing Rate: {row[0]:.2f}%')


Reading Passing Rate: 85.81%


In [697]:
# Calculate the overall passing rate (average of the math and reading passing rate)
query = """
SELECT (((((SUM(CASE WHEN student.math_score >= 70 THEN 1 ELSE 0 END) * 1.0) / (SUM(DISTINCT(school.size) ))) * 100) +
    ((SUM(CASE WHEN student.reading_score >= 70 THEN 1 ELSE 0 END) * 1.0) / (SUM(DISTINCT(school.size) ))) * 100) / 2)
    AS avg_passing_rate
FROM student 
JOIN school on school.school_name = student.school_name
"""
avg_passing_rate = con.execute(query).fetchall()
for school in avg_passing_rate:
    print(f'Passing Rate: {school[0]:.2f}%')



Passing Rate: 80.39%


### Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [455]:
#  Sort and display the worst five schools in overall passing rate

query = """
WITH bottom_5_schools AS 
(SELECT school_name, 
    ((((COUNT(CASE WHEN math_score >= 70 THEN 1 END) * 1.0) / (SELECT SUM(size) FROM school WHERE school_name = student.school_name) * 100) +
    ((COUNT(CASE WHEN reading_score >= 70 THEN 1 END) * 1.0) / (SELECT SUM(size) FROM school WHERE school_name = student.school_name) * 100)) / 2) 
    AS avg_passing_rate
    FROM student
    GROUP BY school_name
    ORDER BY avg_passing_rate ASC
    LIMIT 5)
SELECT 
    school_name, 
    avg_passing_rate
FROM bottom_5_schools
ORDER BY avg_passing_rate ASC
"""
avg_passing_rate = con.execute(query).fetchall()
for school in avg_passing_rate:
    print(f'School: {school[0]}, Passing Rate: {school[1]:.2f}%')


School: Rodriguez High School, Passing Rate: 73.29%
School: Figueroa High School, Passing Rate: 73.36%
School: Huang High School, Passing Rate: 73.50%
School: Johnson High School, Passing Rate: 73.64%
School: Ford High School, Passing Rate: 73.80%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [458]:
# Create table that lists the average math score for each school of each grade level.
query = """
SELECT grade, school.school_name, AVG(math_score) as avg_math_score 
FROM student
JOIN school ON school.school_name = student.school_name
GROUP BY grade, school.school_name
ORDER BY school.school_name , grade
"""
avg_math_scores = con.execute(query).fetchall()
for row in avg_math_scores:
    print(f'School: {row[1]}, Grade: {row[0]}, Average Math Score: {row[2]:.2f}')



School: Bailey High School, Grade: 10th, Average Math Score: 77.00
School: Bailey High School, Grade: 11th, Average Math Score: 77.52
School: Bailey High School, Grade: 12th, Average Math Score: 76.49
School: Bailey High School, Grade: 9th, Average Math Score: 77.08
School: Cabrera High School, Grade: 10th, Average Math Score: 83.15
School: Cabrera High School, Grade: 11th, Average Math Score: 82.77
School: Cabrera High School, Grade: 12th, Average Math Score: 83.28
School: Cabrera High School, Grade: 9th, Average Math Score: 83.09
School: Figueroa High School, Grade: 10th, Average Math Score: 76.54
School: Figueroa High School, Grade: 11th, Average Math Score: 76.88
School: Figueroa High School, Grade: 12th, Average Math Score: 77.15
School: Figueroa High School, Grade: 9th, Average Math Score: 76.40
School: Ford High School, Grade: 10th, Average Math Score: 77.67
School: Ford High School, Grade: 11th, Average Math Score: 76.92
School: Ford High School, Grade: 12th, Average Math Score

In [459]:
# Calculate the average math score for 9th grade in each school
query = """
SELECT grade, school.school_name, AVG(math_score) as avg_math_score 
FROM student
JOIN school ON school.school_name = student.school_name
WHERE grade = '9th'
GROUP BY grade, school.school_name
ORDER BY school.school_name , grade
"""
avg_math_scores = con.execute(query).fetchall()
for row in avg_math_scores:
    print(f'School: {row[1]}, Grade: {row[0]}, Average Math Score: {row[2]:.2f}')


School: Bailey High School, Grade: 9th, Average Math Score: 77.08
School: Cabrera High School, Grade: 9th, Average Math Score: 83.09
School: Figueroa High School, Grade: 9th, Average Math Score: 76.40
School: Ford High School, Grade: 9th, Average Math Score: 77.36
School: Griffin High School, Grade: 9th, Average Math Score: 82.04
School: Hernandez High School, Grade: 9th, Average Math Score: 77.44
School: Holden High School, Grade: 9th, Average Math Score: 83.79
School: Huang High School, Grade: 9th, Average Math Score: 77.03
School: Johnson High School, Grade: 9th, Average Math Score: 77.19
School: Pena High School, Grade: 9th, Average Math Score: 83.63
School: Rodriguez High School, Grade: 9th, Average Math Score: 76.86
School: Shelton High School, Grade: 9th, Average Math Score: 83.42
School: Thomas High School, Grade: 9th, Average Math Score: 83.59
School: Wilson High School, Grade: 9th, Average Math Score: 83.09
School: Wright High School, Grade: 9th, Average Math Score: 83.26


In [460]:
# Calculate the average math score for 10th grade in each school
query = """
SELECT grade, school.school_name, AVG(math_score) as avg_math_score 
FROM student
JOIN school ON school.school_name = student.school_name
WHERE grade = '10th'
GROUP BY grade, school.school_name
ORDER BY school.school_name , grade
"""
avg_math_scores = con.execute(query).fetchall()
for row in avg_math_scores:
    print(f'School: {row[1]}, Grade: {row[0]}, Average Math Score: {row[2]:.2f}')



School: Bailey High School, Grade: 10th, Average Math Score: 77.00
School: Cabrera High School, Grade: 10th, Average Math Score: 83.15
School: Figueroa High School, Grade: 10th, Average Math Score: 76.54
School: Ford High School, Grade: 10th, Average Math Score: 77.67
School: Griffin High School, Grade: 10th, Average Math Score: 84.23
School: Hernandez High School, Grade: 10th, Average Math Score: 77.34
School: Holden High School, Grade: 10th, Average Math Score: 83.43
School: Huang High School, Grade: 10th, Average Math Score: 75.91
School: Johnson High School, Grade: 10th, Average Math Score: 76.69
School: Pena High School, Grade: 10th, Average Math Score: 83.37
School: Rodriguez High School, Grade: 10th, Average Math Score: 76.61
School: Shelton High School, Grade: 10th, Average Math Score: 82.92
School: Thomas High School, Grade: 10th, Average Math Score: 83.09
School: Wilson High School, Grade: 10th, Average Math Score: 83.72
School: Wright High School, Grade: 10th, Average Math S

In [461]:
# Calculate the average math score for 11th grade in each school
query = """
SELECT grade, school.school_name, AVG(math_score) as avg_math_score 
FROM student
JOIN school ON school.school_name = student.school_name
WHERE grade = '11th'
GROUP BY grade, school.school_name
ORDER BY school.school_name , grade
"""
avg_math_scores = con.execute(query).fetchall()
for row in avg_math_scores:
    print(f'School: {row[1]}, Grade: {row[0]}, Average Math Score: {row[2]:.2f}')



School: Bailey High School, Grade: 11th, Average Math Score: 77.52
School: Cabrera High School, Grade: 11th, Average Math Score: 82.77
School: Figueroa High School, Grade: 11th, Average Math Score: 76.88
School: Ford High School, Grade: 11th, Average Math Score: 76.92
School: Griffin High School, Grade: 11th, Average Math Score: 83.84
School: Hernandez High School, Grade: 11th, Average Math Score: 77.14
School: Holden High School, Grade: 11th, Average Math Score: 85.00
School: Huang High School, Grade: 11th, Average Math Score: 76.45
School: Johnson High School, Grade: 11th, Average Math Score: 77.49
School: Pena High School, Grade: 11th, Average Math Score: 84.33
School: Rodriguez High School, Grade: 11th, Average Math Score: 76.40
School: Shelton High School, Grade: 11th, Average Math Score: 83.38
School: Thomas High School, Grade: 11th, Average Math Score: 83.50
School: Wilson High School, Grade: 11th, Average Math Score: 83.20
School: Wright High School, Grade: 11th, Average Math S

In [462]:
# Calculate the average math score for 12th grade in each school
query = """
SELECT grade, school.school_name, AVG(math_score) as avg_math_score 
FROM student
JOIN school ON school.school_name = student.school_name
WHERE grade = '12th'
GROUP BY grade, school.school_name
ORDER BY school.school_name , grade
"""
avg_math_scores = con.execute(query).fetchall()
for row in avg_math_scores:
    print(f'School: {row[1]}, Grade: {row[0]}, Average Math Score: {row[2]:.2f}')



School: Bailey High School, Grade: 12th, Average Math Score: 76.49
School: Cabrera High School, Grade: 12th, Average Math Score: 83.28
School: Figueroa High School, Grade: 12th, Average Math Score: 77.15
School: Ford High School, Grade: 12th, Average Math Score: 76.18
School: Griffin High School, Grade: 12th, Average Math Score: 83.36
School: Hernandez High School, Grade: 12th, Average Math Score: 77.19
School: Holden High School, Grade: 12th, Average Math Score: 82.86
School: Huang High School, Grade: 12th, Average Math Score: 77.23
School: Johnson High School, Grade: 12th, Average Math Score: 76.86
School: Pena High School, Grade: 12th, Average Math Score: 84.12
School: Rodriguez High School, Grade: 12th, Average Math Score: 77.69
School: Shelton High School, Grade: 12th, Average Math Score: 83.78
School: Thomas High School, Grade: 12th, Average Math Score: 83.50
School: Wilson High School, Grade: 12th, Average Math Score: 83.04
School: Wright High School, Grade: 12th, Average Math S

### Reading Score by Grade 

* Perform the same operations as above for reading scores

In [465]:
# Create table that lists the average reading score for each school of each grade level.
query = """
SELECT grade, school.school_name, AVG(reading_score) as avg_reading_score 
FROM student
JOIN school ON school.school_name = student.school_name
GROUP BY grade, school.school_name
ORDER BY school.school_name , grade
"""
avg_reading_scores = con.execute(query).fetchall()
for row in avg_reading_scores:
    print(f'School: {row[1]}, Grade: {row[0]}, Average Reading Score: {row[2]:.2f}')


School: Bailey High School, Grade: 10th, Average Reading Score: 80.91
School: Bailey High School, Grade: 11th, Average Reading Score: 80.95
School: Bailey High School, Grade: 12th, Average Reading Score: 80.91
School: Bailey High School, Grade: 9th, Average Reading Score: 81.30
School: Cabrera High School, Grade: 10th, Average Reading Score: 84.25
School: Cabrera High School, Grade: 11th, Average Reading Score: 83.79
School: Cabrera High School, Grade: 12th, Average Reading Score: 84.29
School: Cabrera High School, Grade: 9th, Average Reading Score: 83.68
School: Figueroa High School, Grade: 10th, Average Reading Score: 81.41
School: Figueroa High School, Grade: 11th, Average Reading Score: 80.64
School: Figueroa High School, Grade: 12th, Average Reading Score: 81.38
School: Figueroa High School, Grade: 9th, Average Reading Score: 81.20
School: Ford High School, Grade: 10th, Average Reading Score: 81.26
School: Ford High School, Grade: 11th, Average Reading Score: 80.40
School: Ford Hi

In [466]:
# Calculate the average reading score for 9th grade in each school
query = """
SELECT grade, school.school_name, AVG(reading_score) as avg_reading_score 
FROM student
JOIN school ON school.school_name = student.school_name
WHERE grade = '9th'
GROUP BY grade, school.school_name
ORDER BY school.school_name , grade
"""
avg_reading_scores = con.execute(query).fetchall()
for row in avg_reading_scores:
    print(f'School: {row[1]}, Grade: {row[0]}, Average Reading Score: {row[2]:.2f}')



School: Bailey High School, Grade: 9th, Average Reading Score: 81.30
School: Cabrera High School, Grade: 9th, Average Reading Score: 83.68
School: Figueroa High School, Grade: 9th, Average Reading Score: 81.20
School: Ford High School, Grade: 9th, Average Reading Score: 80.63
School: Griffin High School, Grade: 9th, Average Reading Score: 83.37
School: Hernandez High School, Grade: 9th, Average Reading Score: 80.87
School: Holden High School, Grade: 9th, Average Reading Score: 83.68
School: Huang High School, Grade: 9th, Average Reading Score: 81.29
School: Johnson High School, Grade: 9th, Average Reading Score: 81.26
School: Pena High School, Grade: 9th, Average Reading Score: 83.81
School: Rodriguez High School, Grade: 9th, Average Reading Score: 80.99
School: Shelton High School, Grade: 9th, Average Reading Score: 84.12
School: Thomas High School, Grade: 9th, Average Reading Score: 83.73
School: Wilson High School, Grade: 9th, Average Reading Score: 83.94
School: Wright High School,

In [467]:
# Calculate the average reading score for 10th grade in each school
query = """
SELECT grade, school.school_name, AVG(reading_score) as avg_reading_score 
FROM student
JOIN school ON school.school_name = student.school_name
WHERE grade = '10th'
GROUP BY grade, school.school_name
ORDER BY school.school_name , grade
"""
avg_reading_scores = con.execute(query).fetchall()
for row in avg_reading_scores:
    print(f'School: {row[1]}, Grade: {row[0]}, Average Reading Score: {row[2]:.2f}')




School: Bailey High School, Grade: 10th, Average Reading Score: 80.91
School: Cabrera High School, Grade: 10th, Average Reading Score: 84.25
School: Figueroa High School, Grade: 10th, Average Reading Score: 81.41
School: Ford High School, Grade: 10th, Average Reading Score: 81.26
School: Griffin High School, Grade: 10th, Average Reading Score: 83.71
School: Hernandez High School, Grade: 10th, Average Reading Score: 80.66
School: Holden High School, Grade: 10th, Average Reading Score: 83.32
School: Huang High School, Grade: 10th, Average Reading Score: 81.51
School: Johnson High School, Grade: 10th, Average Reading Score: 80.77
School: Pena High School, Grade: 10th, Average Reading Score: 83.61
School: Rodriguez High School, Grade: 10th, Average Reading Score: 80.63
School: Shelton High School, Grade: 10th, Average Reading Score: 83.44
School: Thomas High School, Grade: 10th, Average Reading Score: 84.25
School: Wilson High School, Grade: 10th, Average Reading Score: 84.02
School: Wrigh

In [468]:
# Calculate the average reading score for 11th grade in each school
query = """
SELECT grade, school.school_name, AVG(reading_score) as avg_reading_score 
FROM student
JOIN school ON school.school_name = student.school_name
WHERE grade = '11th'
GROUP BY grade, school.school_name
ORDER BY school.school_name , grade
"""
avg_reading_scores = con.execute(query).fetchall()
for row in avg_reading_scores:
    print(f'School: {row[1]}, Grade: {row[0]}, Average Reading Score: {row[2]:.2f}')




School: Bailey High School, Grade: 11th, Average Reading Score: 80.95
School: Cabrera High School, Grade: 11th, Average Reading Score: 83.79
School: Figueroa High School, Grade: 11th, Average Reading Score: 80.64
School: Ford High School, Grade: 11th, Average Reading Score: 80.40
School: Griffin High School, Grade: 11th, Average Reading Score: 84.29
School: Hernandez High School, Grade: 11th, Average Reading Score: 81.40
School: Holden High School, Grade: 11th, Average Reading Score: 83.82
School: Huang High School, Grade: 11th, Average Reading Score: 81.42
School: Johnson High School, Grade: 11th, Average Reading Score: 80.62
School: Pena High School, Grade: 11th, Average Reading Score: 84.34
School: Rodriguez High School, Grade: 11th, Average Reading Score: 80.86
School: Shelton High School, Grade: 11th, Average Reading Score: 84.37
School: Thomas High School, Grade: 11th, Average Reading Score: 83.59
School: Wilson High School, Grade: 11th, Average Reading Score: 83.76
School: Wrigh

In [469]:
# Calculate the average reading score for 12th grade in each school
query = """
SELECT grade, school.school_name, AVG(reading_score) as avg_reading_score 
FROM student
JOIN school ON school.school_name = student.school_name
WHERE grade = '12th'
GROUP BY grade, school.school_name
ORDER BY school.school_name , grade
"""
avg_reading_scores = con.execute(query).fetchall()
for row in avg_reading_scores:
    print(f'School: {row[1]}, Grade: {row[0]}, Average Reading Score: {row[2]:.2f}')




School: Bailey High School, Grade: 12th, Average Reading Score: 80.91
School: Cabrera High School, Grade: 12th, Average Reading Score: 84.29
School: Figueroa High School, Grade: 12th, Average Reading Score: 81.38
School: Ford High School, Grade: 12th, Average Reading Score: 80.66
School: Griffin High School, Grade: 12th, Average Reading Score: 84.01
School: Hernandez High School, Grade: 12th, Average Reading Score: 80.86
School: Holden High School, Grade: 12th, Average Reading Score: 84.70
School: Huang High School, Grade: 12th, Average Reading Score: 80.31
School: Johnson High School, Grade: 12th, Average Reading Score: 81.23
School: Pena High School, Grade: 12th, Average Reading Score: 84.59
School: Rodriguez High School, Grade: 12th, Average Reading Score: 80.38
School: Shelton High School, Grade: 12th, Average Reading Score: 82.78
School: Thomas High School, Grade: 12th, Average Reading Score: 83.83
School: Wilson High School, Grade: 12th, Average Reading Score: 84.32
School: Wrigh

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [472]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [473]:
# Create a new column to show budget per student in each row
query = """
SELECT school.school_name, ((school.budget) / (school.size)) as school_budget 
FROM school 
    
"""
school_budget = con.execute(query).fetchall()
print(f'School Budget per student: {school_budget}')


School Budget per student: [('Huang High School', 655), ('Figueroa High School', 639), ('Shelton High School', 600), ('Hernandez High School', 652), ('Griffin High School', 625), ('Wilson High School', 578), ('Cabrera High School', 582), ('Bailey High School', 628), ('Holden High School', 581), ('Pena High School', 609), ('Wright High School', 583), ('Rodriguez High School', 637), ('Johnson High School', 650), ('Ford High School', 644), ('Thomas High School', 638)]


In [474]:
# Create a new column to define the spending ranges per student

spending_bins = [0, 585, 615, 645, 675]


In [515]:
# Calculate the average math score within each spending range
query = """
SELECT AVG(student.math_score) as avg_math_score, school.school_name, (school.budget) / (school.size) as school_budget,
    CASE
        WHEN (school.budget) / (school.size) <= 585 THEN ('0-585')
        WHEN (school.budget) / (school.size) > 585 AND (school.budget) / (school.size) <=615 THEN ('586-615')
        WHEN (school.budget) / (school.size) > 615 AND (school.budget) / (school.size) <=645 THEN ('616-645')
        WHEN (school.budget) / (school.size) > 645 AND (school.budget) / (school.size) <=675 THEN ('646-675')
        ELSE 'Above 675'       
    END AS budget_category
FROM school
JOIN student ON student.school_name = school.school_name
GROUP BY budget_category, school.school_name
ORDER BY budget_category
"""
school_budget = con.execute(query).fetchall()

for row in school_budget:
    print(f'Budget Category: {row[3]}, School: {row[1]}, Average Math Score: {row[0]:.2f}')

Budget Category: 0-585, School: Cabrera High School, Average Math Score: 83.06
Budget Category: 0-585, School: Holden High School, Average Math Score: 83.80
Budget Category: 0-585, School: Wilson High School, Average Math Score: 83.27
Budget Category: 0-585, School: Wright High School, Average Math Score: 83.68
Budget Category: 586-615, School: Pena High School, Average Math Score: 83.84
Budget Category: 586-615, School: Shelton High School, Average Math Score: 83.36
Budget Category: 616-645, School: Bailey High School, Average Math Score: 77.05
Budget Category: 616-645, School: Figueroa High School, Average Math Score: 76.71
Budget Category: 616-645, School: Ford High School, Average Math Score: 77.10
Budget Category: 616-645, School: Griffin High School, Average Math Score: 83.35
Budget Category: 616-645, School: Rodriguez High School, Average Math Score: 76.84
Budget Category: 616-645, School: Thomas High School, Average Math Score: 83.42
Budget Category: 646-675, School: Hernandez 

In [513]:
# Calculate the percentage passing rate for math in each spending range
query = """
SELECT ((COUNT(CASE WHEN student.math_score >= 70 THEN 1 END) * 1.0) / ((school.size))) * 100 AS Passing_Rate, school.school_name, (school.budget) / (school.size) as school_budget,
    CASE
        WHEN (school.budget) / (school.size) <= 585 THEN ('0-585')
        WHEN (school.budget) / (school.size) > 585 AND (school.budget) / (school.size) <=615 THEN ('586-615')
        WHEN (school.budget) / (school.size) > 615 AND (school.budget) / (school.size) <=645 THEN ('616-645')
        WHEN (school.budget) / (school.size) > 645 AND (school.budget) / (school.size) <=675 THEN ('646-675')
        ELSE 'Above 675'       
    END AS budget_category
FROM school
JOIN student ON student.school_name = school.school_name
GROUP BY budget_category, school.school_name
ORDER BY budget_category
"""
school_budget = con.execute(query).fetchall()

for row in school_budget:
    print(f'Budget Category: {row[3]}, School: {row[1]}, Passing Rate: {row[0]:.2f}%')




Budget Category: 0-585, School: Cabrera High School, Passing Rate: 94.13%
Budget Category: 0-585, School: Holden High School, Passing Rate: 92.51%
Budget Category: 0-585, School: Wilson High School, Passing Rate: 93.87%
Budget Category: 0-585, School: Wright High School, Passing Rate: 93.33%
Budget Category: 586-615, School: Pena High School, Passing Rate: 94.59%
Budget Category: 586-615, School: Shelton High School, Passing Rate: 93.87%
Budget Category: 616-645, School: Bailey High School, Passing Rate: 66.68%
Budget Category: 616-645, School: Figueroa High School, Passing Rate: 65.99%
Budget Category: 616-645, School: Ford High School, Passing Rate: 68.31%
Budget Category: 616-645, School: Griffin High School, Passing Rate: 93.39%
Budget Category: 616-645, School: Rodriguez High School, Passing Rate: 66.37%
Budget Category: 616-645, School: Thomas High School, Passing Rate: 93.27%
Budget Category: 646-675, School: Hernandez High School, Passing Rate: 66.75%
Budget Category: 646-675, 

In [517]:
# Calculate the percentage passing rate for reading in each spending range
query = """
SELECT ((COUNT(CASE WHEN student.reading_score >= 70 THEN 1 END) * 1.0) / ((school.size))) * 100 AS Passing_Rate, school.school_name, (school.budget) / (school.size) as school_budget,
    CASE
        WHEN (school.budget) / (school.size) <= 585 THEN ('0-585')
        WHEN (school.budget) / (school.size) > 585 AND (school.budget) / (school.size) <=615 THEN ('586-615')
        WHEN (school.budget) / (school.size) > 615 AND (school.budget) / (school.size) <=645 THEN ('616-645')
        WHEN (school.budget) / (school.size) > 645 AND (school.budget) / (school.size) <=675 THEN ('646-675')
        ELSE 'Above 675'       
    END AS budget_category
FROM school
JOIN student ON student.school_name = school.school_name
GROUP BY budget_category, school.school_name
ORDER BY budget_category
"""
school_budget = con.execute(query).fetchall()

for row in school_budget:
    print(f'Budget Category: {row[3]}, School: {row[1]}, Passing Rate: {row[0]:.2f}%')




Budget Category: 0-585, School: Cabrera High School, Passing Rate: 97.04%
Budget Category: 0-585, School: Holden High School, Passing Rate: 96.25%
Budget Category: 0-585, School: Wilson High School, Passing Rate: 96.54%
Budget Category: 0-585, School: Wright High School, Passing Rate: 96.61%
Budget Category: 586-615, School: Pena High School, Passing Rate: 95.95%
Budget Category: 586-615, School: Shelton High School, Passing Rate: 95.85%
Budget Category: 616-645, School: Bailey High School, Passing Rate: 81.93%
Budget Category: 616-645, School: Figueroa High School, Passing Rate: 80.74%
Budget Category: 616-645, School: Ford High School, Passing Rate: 79.30%
Budget Category: 616-645, School: Griffin High School, Passing Rate: 97.14%
Budget Category: 616-645, School: Rodriguez High School, Passing Rate: 80.22%
Budget Category: 616-645, School: Thomas High School, Passing Rate: 97.31%
Budget Category: 646-675, School: Hernandez High School, Passing Rate: 80.86%
Budget Category: 646-675, 

In [527]:
# Calculate the percentage overall passing rate in each spending range
query = """
SELECT (((((COUNT(CASE WHEN math_score >= 70 THEN 1 END) * 1.0) / (school.size))) +
    ((COUNT(CASE WHEN reading_score >= 70 THEN 1 END) * 1.0) / (school.size))) / 2) * 100 AS Passing_Rate, school.school_name, (school.budget) / (school.size) as school_budget,
    CASE
        WHEN (school.budget) / (school.size) <= 585 THEN ('0-585')
        WHEN (school.budget) / (school.size) > 585 AND (school.budget) / (school.size) <=615 THEN ('586-615')
        WHEN (school.budget) / (school.size) > 615 AND (school.budget) / (school.size) <=645 THEN ('616-645')
        WHEN (school.budget) / (school.size) > 645 AND (school.budget) / (school.size) <=675 THEN ('646-675')
        ELSE 'Above 675'       
    END AS budget_category
FROM school
JOIN student ON student.school_name = school.school_name
GROUP BY budget_category, school.school_name
ORDER BY budget_category
"""
school_budget = con.execute(query).fetchall()

for row in school_budget:
    print(f'Budget Category: {row[3]}, School: {row[1]}, Passing Rate: {row[0]:.2f}%')




Budget Category: 0-585, School: Cabrera High School, Passing Rate: 95.59%
Budget Category: 0-585, School: Holden High School, Passing Rate: 94.38%
Budget Category: 0-585, School: Wilson High School, Passing Rate: 95.20%
Budget Category: 0-585, School: Wright High School, Passing Rate: 94.97%
Budget Category: 586-615, School: Pena High School, Passing Rate: 95.27%
Budget Category: 586-615, School: Shelton High School, Passing Rate: 94.86%
Budget Category: 616-645, School: Bailey High School, Passing Rate: 74.31%
Budget Category: 616-645, School: Figueroa High School, Passing Rate: 73.36%
Budget Category: 616-645, School: Ford High School, Passing Rate: 73.80%
Budget Category: 616-645, School: Griffin High School, Passing Rate: 95.27%
Budget Category: 616-645, School: Rodriguez High School, Passing Rate: 73.29%
Budget Category: 616-645, School: Thomas High School, Passing Rate: 95.29%
Budget Category: 646-675, School: Hernandez High School, Passing Rate: 73.81%
Budget Category: 646-675, 

### Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [ ]:
# Create a new column for the bin groups
size_bins = [0, 1000, 2000, 5000]

Look for the total count of test scores that pass 70% or higher




In [543]:
# math_pass_size
query = """
SELECT AVG(student.math_score) as avg_math_score, school.school_name, (school.size) as school_size,
    CASE
        WHEN (school.size) <= 1000 THEN 'Small (0-1000)'
        WHEN (school.size) > 1000 AND (school.size) <=2000 THEN 'Medium (1001-2000)'
        WHEN (school.size) > 2000 AND (school.size) <=5000 THEN 'Large (2001-5000)'
        ELSE 'X-Large (5001+)'       
    END AS size_category
FROM school
JOIN student ON student.school_name = school.school_name
GROUP BY size_category, school.school_name, school.size
ORDER BY size_category 
"""
school_size = con.execute(query).fetchall()

for row in school_size:
    print(f'Size Category: {row[3]}, School: {row[1]}, Average Math Score: {row[0]:.2f}')



Size Category: Large (2001-5000), School: Bailey High School, Average Math Score: 77.05
Size Category: Large (2001-5000), School: Figueroa High School, Average Math Score: 76.71
Size Category: Large (2001-5000), School: Ford High School, Average Math Score: 77.10
Size Category: Large (2001-5000), School: Hernandez High School, Average Math Score: 77.29
Size Category: Large (2001-5000), School: Huang High School, Average Math Score: 76.63
Size Category: Large (2001-5000), School: Johnson High School, Average Math Score: 77.07
Size Category: Large (2001-5000), School: Rodriguez High School, Average Math Score: 76.84
Size Category: Large (2001-5000), School: Wilson High School, Average Math Score: 83.27
Size Category: Medium (1001-2000), School: Cabrera High School, Average Math Score: 83.06
Size Category: Medium (1001-2000), School: Griffin High School, Average Math Score: 83.35
Size Category: Medium (1001-2000), School: Shelton High School, Average Math Score: 83.36
Size Category: Mediu

In [545]:
# read_pass_size
query = """
SELECT AVG(student.reading_score) as avg_reading_score, school.school_name, (school.size) as school_size,
    CASE
        WHEN (school.size) <= 1000 THEN 'Small (0-1000)'
        WHEN (school.size) > 1000 AND (school.size) <=2000 THEN 'Medium (1001-2000)'
        WHEN (school.size) > 2000 AND (school.size) <=5000 THEN 'Large (2001-5000)'
        ELSE 'X-Large (5001+)'       
    END AS size_category
FROM school
JOIN student ON student.school_name = school.school_name
GROUP BY size_category, school.school_name, school.size
ORDER BY size_category 
"""
school_size = con.execute(query).fetchall()

for row in school_size:
    print(f'Size Category: {row[3]}, School: {row[1]}, Average Math Score: {row[0]:.2f}')



Size Category: Large (2001-5000), School: Bailey High School, Average Math Score: 81.03
Size Category: Large (2001-5000), School: Figueroa High School, Average Math Score: 81.16
Size Category: Large (2001-5000), School: Ford High School, Average Math Score: 80.75
Size Category: Large (2001-5000), School: Hernandez High School, Average Math Score: 80.93
Size Category: Large (2001-5000), School: Huang High School, Average Math Score: 81.18
Size Category: Large (2001-5000), School: Johnson High School, Average Math Score: 80.97
Size Category: Large (2001-5000), School: Rodriguez High School, Average Math Score: 80.74
Size Category: Large (2001-5000), School: Wilson High School, Average Math Score: 83.99
Size Category: Medium (1001-2000), School: Cabrera High School, Average Math Score: 83.98
Size Category: Medium (1001-2000), School: Griffin High School, Average Math Score: 83.82
Size Category: Medium (1001-2000), School: Shelton High School, Average Math Score: 83.73
Size Category: Mediu

In [547]:
# Calculate the overall passing rate for different school size
query = """
SELECT (((((COUNT(CASE WHEN math_score >= 70 THEN 1 END) * 1.0) / (school.size))) +
    ((COUNT(CASE WHEN reading_score >= 70 THEN 1 END) * 1.0) / (school.size))) / 2) * 100 AS Passing_Rate, school.school_name, (school.size) as school_size,
    CASE
        WHEN (school.size) <= 1000 THEN 'Small (0-1000)'
        WHEN (school.size) > 1000 AND (school.size) <=2000 THEN 'Medium (1001-2000)'
        WHEN (school.size) > 2000 AND (school.size) <=5000 THEN 'Large (2001-5000)'
        ELSE 'X-Large (5001+)'       
    END AS size_category
FROM school
JOIN student ON student.school_name = school.school_name
GROUP BY size_category, school.school_name, school.size
ORDER BY size_category
"""
school_size = con.execute(query).fetchall()

for row in school_size:
    print(f'Size Category: {row[3]}, School: {row[1]}, Passing Rate: {row[0]:.2f}%')



Size Category: Large (2001-5000), School: Bailey High School, Passing Rate: 74.31%
Size Category: Large (2001-5000), School: Figueroa High School, Passing Rate: 73.36%
Size Category: Large (2001-5000), School: Ford High School, Passing Rate: 73.80%
Size Category: Large (2001-5000), School: Hernandez High School, Passing Rate: 73.81%
Size Category: Large (2001-5000), School: Huang High School, Passing Rate: 73.50%
Size Category: Large (2001-5000), School: Johnson High School, Passing Rate: 73.64%
Size Category: Large (2001-5000), School: Rodriguez High School, Passing Rate: 73.29%
Size Category: Large (2001-5000), School: Wilson High School, Passing Rate: 95.20%
Size Category: Medium (1001-2000), School: Cabrera High School, Passing Rate: 95.59%
Size Category: Medium (1001-2000), School: Griffin High School, Passing Rate: 95.27%
Size Category: Medium (1001-2000), School: Shelton High School, Passing Rate: 94.86%
Size Category: Medium (1001-2000), School: Thomas High School, Passing Rate

### Scores by School Type

* Perform the same operations as above, based on school type.

In [ ]:
# Create bins and groups, school type {'Charter', 'District'}

type_bins = ['Charter', 'District']


Find counts of the passing 70 or higher score for the both test


In [557]:
# math pass size
query = """
SELECT AVG(student.math_score) as avg_math_score, school.school_name, (school.type) as school_type,
    CASE
        WHEN (school.type) = 'District' THEN 'District'
        WHEN (school.type) = 'Charter' THEN 'Charter'
        ELSE 'Custom'       
    END AS school_category
FROM school
JOIN student ON student.school_name = school.school_name
GROUP BY school_category, school.school_name, school.type
ORDER BY school_category 
"""
school_type = con.execute(query).fetchall()

for row in school_type:
    print(f'School Category: {row[3]}, School: {row[1]}, Average Math Score: {row[0]:.2f}')



School Category: Charter, School: Cabrera High School, Average Math Score: 83.06
School Category: Charter, School: Griffin High School, Average Math Score: 83.35
School Category: Charter, School: Holden High School, Average Math Score: 83.80
School Category: Charter, School: Pena High School, Average Math Score: 83.84
School Category: Charter, School: Shelton High School, Average Math Score: 83.36
School Category: Charter, School: Thomas High School, Average Math Score: 83.42
School Category: Charter, School: Wilson High School, Average Math Score: 83.27
School Category: Charter, School: Wright High School, Average Math Score: 83.68
School Category: District, School: Bailey High School, Average Math Score: 77.05
School Category: District, School: Figueroa High School, Average Math Score: 76.71
School Category: District, School: Ford High School, Average Math Score: 77.10
School Category: District, School: Hernandez High School, Average Math Score: 77.29
School Category: District, Schoo

In [559]:
# reading pass size
query = """
SELECT AVG(student.reading_score) as avg_reading_score, school.school_name, (school.type) as school_type,
    CASE
        WHEN (school.type) = 'District' THEN 'District'
        WHEN (school.type) = 'Charter' THEN 'Charter'
        ELSE 'Custom'       
    END AS school_category
FROM school
JOIN student ON student.school_name = school.school_name
GROUP BY school_category, school.school_name, school.type
ORDER BY school_category 
"""
school_type = con.execute(query).fetchall()

for row in school_type:
    print(f'School Category: {row[3]}, School: {row[1]}, Average Reading Score: {row[0]:.2f}')


School Category: Charter, School: Cabrera High School, Average Reading Score: 83.98
School Category: Charter, School: Griffin High School, Average Reading Score: 83.82
School Category: Charter, School: Holden High School, Average Reading Score: 83.81
School Category: Charter, School: Pena High School, Average Reading Score: 84.04
School Category: Charter, School: Shelton High School, Average Reading Score: 83.73
School Category: Charter, School: Thomas High School, Average Reading Score: 83.85
School Category: Charter, School: Wilson High School, Average Reading Score: 83.99
School Category: Charter, School: Wright High School, Average Reading Score: 83.95
School Category: District, School: Bailey High School, Average Reading Score: 81.03
School Category: District, School: Figueroa High School, Average Reading Score: 81.16
School Category: District, School: Ford High School, Average Reading Score: 80.75
School Category: District, School: Hernandez High School, Average Reading Score: 80

In [565]:
# Calculate the overall passing rate

query = """
SELECT (((((COUNT(CASE WHEN math_score >= 70 THEN 1 END) * 1.0) / (school.size))) +
    ((COUNT(CASE WHEN reading_score >= 70 THEN 1 END) * 1.0) / (school.size))) / 2) * 100 AS Passing_Rate, school.school_name, (school.type) as school_type,
    CASE
        WHEN (school.type) = 'District' THEN 'District'
        WHEN (school.type) = 'Charter' THEN 'Charter'
        ELSE 'Custom'       
    END AS school_category
FROM school
JOIN student ON student.school_name = school.school_name
GROUP BY school_category, school.school_name, school.type
ORDER BY school_category 
"""
school_type = con.execute(query).fetchall()

for row in school_type:
    print(f'School Category: {row[3]}, School: {row[1]}, Average Reading Score: {row[0]:.2f}')


School Category: Charter, School: Cabrera High School, Average Reading Score: 95.59
School Category: Charter, School: Griffin High School, Average Reading Score: 95.27
School Category: Charter, School: Holden High School, Average Reading Score: 94.38
School Category: Charter, School: Pena High School, Average Reading Score: 95.27
School Category: Charter, School: Shelton High School, Average Reading Score: 94.86
School Category: Charter, School: Thomas High School, Average Reading Score: 95.29
School Category: Charter, School: Wilson High School, Average Reading Score: 95.20
School Category: Charter, School: Wright High School, Average Reading Score: 94.97
School Category: District, School: Bailey High School, Average Reading Score: 74.31
School Category: District, School: Figueroa High School, Average Reading Score: 73.36
School Category: District, School: Ford High School, Average Reading Score: 73.80
School Category: District, School: Hernandez High School, Average Reading Score: 73